## **Поведение пользователей. Описание проекта**

**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

In [29]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold, cross_val_score, HalvingGridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.experimental import enable_halving_search_cv
from fast_ml.model_development import train_valid_test_split

import warnings
warnings.filterwarnings('ignore')


Wall time: 1e+03 µs


In [31]:
display(df)


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [ ]:
# Разделение исходных данных на обучающую, валидационную и тестовую выборки.
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, 
    target = 'is_ultra', train_size=0.6, valid_size=0.2, test_size=0.2)

## Исследование качества разных моделей, изменение гиперпараметров. 

**Вариант 1.RandomizedSearchCV:**

In [ ]:
%%time

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]

max_features = ['auto', 'sqrt']

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

rf_grid = {'n_estimators': n_estimators,
           'max_features': max_features,
           'max_depth': max_depth,
           'min_samples_split': min_samples_split,
           'min_samples_leaf': min_samples_leaf}

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

model = RandomizedSearchCV(estimator = RandomForestClassifier(), 
                   param_distributions = rf_grid, 
                   cv = cv, n_iter = 100, scoring='accuracy', n_jobs=-1)

result = model.fit(X_train, y_train)

print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


**Вариант 2. HalvingGridSearchCV:**

In [38]:
%%time
param_grid = {
"max_depth": [3, 4, 7, 10, 25],
"gamma": [0.5, 1, 5, 10, 25],
"min_child_weight": [1, 3, 5, 10, 25],
"reg_lambda": [5, 10, 50, 100, 300],
"scale_pos_weight": [1, 3, 5, 10, 25]
}
# Grid Search CV implementation
xgb_cl = xgb.XGBClassifier(objective="binary:logistic")
model = HalvingGridSearchCV(xgb_cl, param_grid, scoring="roc_auc", n_jobs=-1, min_resources="exhaust", factor=3)
model.fit(X_train, y_train)
# Return set of parameters with the best performance

result = model.fit(X_train, y_train)

print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Best Score: 0.8268182981783287
Best Hyperparameters: {'gamma': 0.5, 'max_depth': 3, 'min_child_weight': 1, 'reg_lambda': 5, 'scale_pos_weight': 1}
Accuracy: 80.40%
Wall time: 9min 12s


## Проверка качества модели на тестовой выборке.

In [36]:
%%time
clf = DummyClassifier(strategy='most_frequent', random_state=0)
clf.fit(X_train, y_train)
#DummyClassifier(random_state=0, strategy='most_frequent')
clf.score(X_test, y_test)


Wall time: 2 ms


0.6702954898911353

In [37]:
df['is_ultra'].value_counts() / df.shape[0]


0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

## Заключение
- Разделили исходные данные на обучающую, валидационную и тестовую выборки (60-20-20).
- Построили модель с максимально большим значением accuracy. Довели долю правильных ответов более 0.75 и проверили accuracy на тестовой выборке.